In [2]:
import pandas as pd
import os
import re
import pymorphy2

In [49]:
"""""""""
4)Признаки, полученные из базового всеобъемлющего корпуса, такие как частота вхождения слова в различные документы из корпуса.
5)Признаки, полученные из контекста слова, такие как характеристики предыдущего и следующего слова, включая то, является ли предыдущее / следующее слово: началом предложения, цифрой или числом, английским словом или медицинским термином, пунктуацией, или конец предложения.


на завтра: доизвлечь признаки, прикрутить grid_search, доразметить данные и пообучать на большем количестве, попробовать дропнуть лишние токены

"""""""""

'\n3)Признаки, полученные из баз знаний, такие как определение того, является ли слово английским словом или медицинским термином.\n4)Признаки, полученные из базового всеобъемлющего корпуса, такие как частота вхождения слова в различные документы из корпуса.\n5)Признаки, полученные из контекста слова, такие как характеристики предыдущего и следующего слова, включая то, является ли предыдущее / следующее слово: началом предложения, цифрой или числом, английским словом или медицинским термином, пунктуацией, или конец предложения.\n\n\nна завтра: доизвлечь признаки, прикрутить grid_search, доразметить данные и пообучать на большем количестве, попробовать дропнуть лишние токены\n\n'

In [3]:
#номинальный категоральный атрибут
def special_symbols(df):
    feature = []
    for token in df["token"]:
        ar = re.findall('\W+', token)
        val = 1 if len(ar)>0 else 0
        feature.append(val)
    return feature

In [4]:
#номинальный категоральный атрибут
def letters_numbers_combination(df):
    #0 - ни букв, ни цифр
    #1 - только буквы
    #2 - только цифры
    #3 - комбинация
    feature = []
    for token in df["token"]:
        letters = re.findall('[а-яА-ЯёЁa-zA-Z]', token)
        numbers = re.findall('\d+', token)
        val = 0
        if len(letters)==0 and len(numbers)==0:
            val = 0
        elif len(numbers)==0:
            val = 1
        elif len(letters)==0:
            val = 2
        else:
            val = 3
        feature.append(val)
    return feature

In [6]:
#порядковый категориальный атрибут
def get_token_length(df):
    feature = []
    for token in df['token']:
        feature.append(len(token))
    return feature

In [7]:
#неприрывный числовой атрибут
def upper_letters_rate(df):
    #отношение количества заглавных букв к длине слова
    upper_russian = 'аоиеёэыуюябвгджзйклмнпрстфхцчшщьъ'.upper()
    upper_english = 'aeioubcdfghjklmnpqrstvwxyz'.upper()
    feature = []
    for token in df['token']:
        uppers = re.findall('[{0}{1}]'.format(upper_russian, upper_english), token)
        val = 0 if len(uppers)==0 else len(uppers)/len(token)
        feature.append(val)
    return feature

In [8]:
#номинальный категоральный атрибут
def upper_letters_inside(df):
    #0 - нет заглавных букв внутри слова
    #1 - есть заглавные буквы внутри слова
    upper_russian = 'аоиеёэыуюябвгджзйклмнпрстфхцчшщьъ'.upper()
    upper_english = 'aeioubcdfghjklmnpqrstvwxyz'.upper()
    feature = []
    for token in df['token']:
        #if len(token)==1 and token[0] in upper_russian+upper_english:
         #   feature.append(1)
        for i in range(1, len(token)):
            if token[i].isupper():
                feature.append(1)
                break
        else:
            feature.append(0)
    return feature

In [9]:
def create_array_from_file(file):
    array = []
    for l in file:
        k = re.split('\s+', l)
        array.append(k[0])
    return array

In [10]:
#номинальный категоральный атрибут
def is_in_dictionary(df):
    rus_dict = open("../dictionaries/dictionary.txt", 'r', encoding='utf-8')
    rus_dict_array = create_array_from_file(rus_dict)
    feature = []
    morph = pymorphy2.MorphAnalyzer()
    for token in df['token']:
        p = morph.parse(token)[0]
        norm = p.normal_form
        is_word = 0
        l = 0
        r = len(rus_dict_array)
        while(l < r-1):
            m = (l+r)//2
            if rus_dict_array[m] <= norm:
                l = m
            else:
                r = m
        is_word = rus_dict_array[l]==norm
        feature.append(int(is_word))
    return feature

In [44]:
df = pd.read_excel("../dataset/ready_data/1.xlsx")

In [45]:
df.head()

,Unnamed: 0,token,is_abbreviation
0,0,Недостаточность,0
1,1,и,0
2,2,дефицит,0
3,3,витамина,0
4,4,D,0


In [ ]:
def process_dataframe(filename):
    path = "../dataset/ready_data/"
    df = pd.read_excel(path+filename)
    df['special_symbols'] = special_symbols(df)
    df['letters_numbers_combination'] = letters_numbers_combination(df)
    df['vowels_consonants_combination'] = vowels_consonants_combination(df)
    df['get_token_length'] = get_token_length(df)
    df['upper_letters_rate'] = upper_letters_rate(df)
    df['upper_letters_inside'] = upper_letters_inside(df)
    df['is_in_dictionary'] = is_in_dictionary(df)
    new_filename = os.path.basename(filename)[0]+"_with_features.csv"
    df.to_csv(path+new_filename)

In [ ]:
process_dataframe("merges_data.xlsx")
process_dataframe("6_without_stopwords.xlsx")